# Lab journal for the Project 4 "Tardigrades: from genestealers to space marines"

> *by Anna Chechenina*

Downloading the assembled genome:

In [ ]:
!mkdir raw_data
!cd raw_data
!wget ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz

--2022-12-15 23:22:48--  http://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 165.112.9.230, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz [following]
--2022-12-15 23:22:49--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/949/185/GCA_001949185.1_Rvar_4.0/GCA_001949185.1_Rvar_4.0_genomic.fna.gz
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17621408 (17M) [application/x-gzip]
Saving to: ‘GCA_001949185.1_Rvar_4.0_genomic.fna.gz’

GCA_001949185.1_Rva 100%[===================>]  16.80M  13.5M

Getting the fasta file with all proteins for all genes predicted with AUGUST (https://math-inf.uni-greifswald.de/en/department/about-us/employees/prof-dr-mario-stanke-english/publications/#c302071 --> book chapters):

In [ ]:
!perl getAnnoFasta.pl augustus.whole.gff

In [ ]:
# number of sequnces
!cat ./augustus.whole.aa | grep '>' | wc -l

16435


That's a lot! So now we have to chose which genes are actually worth to be checked. Since we are dealing with radiation, we'll look for a reparation-related genes. 

We got list of peptides from proteins that are able to bind DNA. We will do a local alignment-based search: create a local database from our protein fasta file and look it up using your peptide sequence file as a query.

## Conda instalation

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:28
🔁 Restarting kernel...


## Diamond run - local alignment

In [ ]:
!conda install -c bioconda diamond

In [ ]:
!diamond --help

In [ ]:
!diamond makedb --in augustus.whole.aa --db tardigrade_db 

diamond v2.0.15.153 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 2
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: augustus.whole.aa
Opening the database file...  [0s]
Loading sequences...  [0.064s]
Masking sequences...  [0.646s]
Writing sequences...  [0.011s]
Hashing sequences...  [0.003s]
Loading sequences...  [0s]
Writing trailer...  [0s]
Closing the input file...  [0s]
Closing the database file...  [0.004s]

Database sequences  16435
  Database letters  7015168
     Database hash  590081acba74e419860a4dfdcf0de70e
        Total time  0.732000s


In [ ]:
!diamond blastp -d tardigrade_db.dmnd -q peptides.fa -f 6 -o diamond_blastp_res.fa --ultra-sensitive                                       

## Blastp run - local alignment (check)

In [ ]:
!conda install -c bioconda blast

In [ ]:
!blastp --help

In [ ]:
!makeblastdb -in augustus.whole.aa -dbtype prot -out tardigrade_db



Building a new DB, current time: 12/16/2022 00:27:48
New DB name:   /content/tardigrade_db
New DB title:  augustus.whole.aa
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 16435 sequences in 0.375116 seconds.




In [ ]:
!blastp -db tardigrade_db -query peptides.fa -outfmt 6 -out blastp_res.fa

We can observe that we cave different number of the output hits for blastp and for diamond. The reason is that diamond have troubles aligning sequences less than 12 amino acids, which is the case in our peptides file. 

So for the further analysis we will use results from blastp (118 lines, 43 identifiers).

## Extracting proteins of interest

Now we will extract the extract proteins of interest from the initial file:

In [ ]:
!tail augustus.whole.aa

FPHCLGYVDGSLIGLQEAPSMCGEDYYTRKGSYAINAMIVCDDLKRIRHVLAGFPGCSHDHRVFMYSKLALNPFKYFSGHEYLLADSAYPVSDIIVPTFK
KPLANSH
>g16433.t1
MHLDADIDESSKEEKPEIVMEYNRTKAGVDTLDQLTGNYSCRRKTSRWPMALFYDILDISTLDAYIIWCEINPGWNSTLPTKRRMFLQDVSKKMMQRQLL
RRSTTPSISFSMKKFVSTVVPRVQVSSDQAVDEMANTSKPKALQKKPTR
>g16434.t1
RIHLLADCHLLKTLNLLQRDREVFPKVDLKSGVIWKVAFFSQALAAMERSVSQKNFGSPVLV
>g16435.t1
MKRPGGQKYHPWSDETKLKSGDVGGANHPWADEAKLNQHPKCAVLSSKLSNVTSMMKFLATGPTGECMSESRYTAFLAGQKNSLGVATKVAGGITSPSQL
CLCKPLLYEEFLTFSNSMVMCETGLDDTCLTRKFK


In [ ]:
!conda install -c bioconda samtools 

In [ ]:
!cat blastp_res.fa | awk '{print $2}' > names.txt # 118 lines, but some preteins repeat

In [ ]:
!uniq -u names.txt > unique_names.txt # 97 unique proteins

We can also get sequenses for them:

In [ ]:
!xargs samtools faidx augustus.whole.aa < unique_names.txt > interest.fa

In [ ]:
!cat ./interest.fa | grep '>' | wc -l

97


In [ ]:
! cat unique_names.txt

g5641.t1
g15153.t1
g5641.t1
g12562.t1
g5616.t1
g15153.t1
g12562.t1
g5616.t1
g702.t1
g15153.t1
g14472.t1
g4106.t1
g7861.t1
g4970.t1
g10513.t1
g11806.t1
g16318.t1
g16368.t1
g3428.t1
g11513.t1
g3428.t1
g12562.t1
g5443.t1
g5502.t1
g15153.t1
g5503.t1
g5467.t1
g12562.t1
g5641.t1
g5616.t1
g5503.t1
g5502.t1
g5510.t1
g12510.t1
g5237.t1
g12510.t1
g11320.t1
g8312.t1
g12510.t1
g5927.t1
g8100.t1
g5641.t1
g15153.t1
g5641.t1
g12562.t1
g5616.t1
g15153.t1
g12562.t1
g5502.t1
g5641.t1
g5503.t1
g1285.t1
g15153.t1
g5616.t1
g12388.t1
g5467.t1
g12562.t1
g5641.t1
g5616.t1
g5503.t1
g5502.t1
g12562.t1
g5502.t1
g5641.t1
g5503.t1
g1285.t1
g15153.t1
g5616.t1
g12388.t1
g5467.t1
g4106.t1
g7861.t1
g4970.t1
g12562.t1
g5443.t1
g5502.t1
g15153.t1
g5503.t1
g5467.t1
g5641.t1
g11960.t1
g702.t1
g10514.t1
g2203.t1
g10514.t1
g5502.t1
g5641.t1
g5503.t1
g1285.t1
g15153.t1
g5616.t1
g12388.t1
g5467.t1
g3428.t1
g11513.t1
g3679.t1
g5237.t1


## Localization prediction

### WoLF PSORT - subcellular localization of proteins

https://wolfpsort.hgc.jp/

See list of annotations here: 
https://docs.google.com/document/d/1MdtVUsnPFN4XjckDrM9Tjh6HzMC3Jl4m_I_mYBQlLRA/edit?usp=sharing 

### TargetP 1.1 Server - also subcellular localization of eukaryotic proteins

https://services.healthtech.dtu.dk/service.php?TargetP-2.0

Results: https://docs.google.com/document/d/1xb_mx1qtqqFWS92RL4FXl1B3IpFY3E95TUeaJEpPZtw/edit?usp=sharing 

In [ ]:
wolf_res = "g5641.t1 details extr: 31, lyso: 1 g15153.t1 details extr: 32 g5641.t1 details extr: 31, lyso: 1 g12562.t1 details extr: 30, lyso: 2 g5616.t1 details extr: 31, mito: 1 g15153.t1 details extr: 32 g12562.t1 details extr: 30, lyso: 2 g5616.t1 details extr: 31, mito: 1 g702.t1 details extr: 29, plas: 2, lyso: 1 g15153.t1 details extr: 32 g14472.t1 details nucl: 28, plas: 2, cyto: 1, cysk: 1 g4106.t1 details E.R.: 14.5, E.R._golg: 9.5, extr: 7, golg: 3.5, lyso: 3, pero: 2, plas: 1, mito: 1 g7861.t1 details nucl: 16, cyto_nucl: 14, cyto: 8, plas: 5, pero: 1, cysk: 1, golg: 1 g4970.t1 details plas: 32 g10513.t1 details nucl: 20, cyto_nucl: 14.5, cyto: 7, extr: 3, E.R.: 1, golg: 1 g11806.t1 details nucl: 18, cyto_nucl: 11.8333, mito: 5, extr: 4, cyto: 3.5, cyto_pero: 2.66667, cysk_plas: 1 g16318.t1 details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1 g16368.t1 details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1 g3428.t1 details mito: 18, cyto: 11, extr: 2, nucl: 1 g11513.t1 details cyto: 17, cyto_nucl: 12.8333, cyto_mito: 9.83333, nucl: 7.5, E.R.: 3, mito: 1.5, plas: 1, pero: 1, golg: 1 g3428.t1 details mito: 18, cyto: 11, extr: 2, nucl: 1 g12562.t1 details extr: 30, lyso: 2 g5443.t1 details extr: 28, nucl: 3, cyto: 1 g5502.t1 details extr: 31, lyso: 1 g15153.t1 details extr: 32 g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 g5467.t1 details extr: 27, plas: 4, mito: 1 g12562.t1 details extr: 30, lyso: 2 g5641.t1 details extr: 31, lyso: 1 g5616.t1 details extr: 31, mito: 1 g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 g5502.t1 details extr: 31, lyso: 1 g5510.t1 details plas: 23, mito: 7, E.R.: 1, golg: 1 g12510.t1 details plas: 29, cyto: 3 g5237.t1 details plas: 24, mito: 8 g12510.t1 details plas: 29, cyto: 3 g11320.t1 details plas: 24.5, extr_plas: 16, extr: 6.5, lyso: 1 g8312.t1 details nucl: 15.5, cyto_nucl: 15.5, cyto: 12.5, mito: 2, plas: 1, golg: 1 g12510.t1 details plas: 29, cyto: 3 g5927.t1 details nucl: 30.5, cyto_nucl: 16.5, cyto: 1.5 g8100.t1 details nucl: 16.5, cyto_nucl: 12.5, cyto: 7.5, plas: 5, extr: 2, E.R.: 1 g5641.t1 details extr: 31, lyso: 1 g15153.t1 details extr: 32 g5641.t1 details extr: 31, lyso: 1 g12562.t1 details extr: 30, lyso: 2 g5616.t1 details extr: 31, mito: 1 g15153.t1 details extr: 32 g12562.t1 details extr: 30, lyso: 2 g5502.t1 details extr: 31, lyso: 1 g5641.t1 details extr: 31, lyso: 1 g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 g1285.t1 details extr: 25, plas: 5, mito: 1, lyso: 1 g15153.t1 details extr: 32 g5616.t1 details extr: 31, mito: 1 g12388.t1 details extr: 25, plas: 4, mito: 2, lyso: 1 g5467.t1 details extr: 27, plas: 4, mito: 1 g12562.t1 details extr: 30, lyso: 2 g5641.t1 details extr: 31, lyso: 1 g5616.t1 details extr: 31, mito: 1 g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 g5502.t1 details extr: 31, lyso: 1 g12562.t1 details extr: 30, lyso: 2 g5502.t1 details extr: 31, lyso: 1 g5641.t1 details extr: 31, lyso: 1 g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 g1285.t1 details extr: 25, plas: 5, mito: 1, lyso: 1 g15153.t1 details extr: 32 g5616.t1 details extr: 31, mito: 1 g12388.t1 details extr: 25, plas: 4, mito: 2, lyso: 1 g5467.t1 details extr: 27, plas: 4, mito: 1 g4106.t1 details E.R.: 14.5, E.R._golg: 9.5, extr: 7, golg: 3.5, lyso: 3, pero: 2, plas: 1, mito: 1 g7861.t1 details nucl: 16, cyto_nucl: 14, cyto: 8, plas: 5, pero: 1, cysk: 1, golg: 1 g4970.t1 details plas: 32 g12562.t1 details extr: 30, lyso: 2 g5443.t1 details extr: 28, nucl: 3, cyto: 1 g5502.t1 details extr: 31, lyso: 1 g15153.t1 details extr: 32 g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 g5467.t1 details extr: 27, plas: 4, mito: 1 g5641.t1 details extr: 31, lyso: 1 g11960.t1 details nucl: 32 g702.t1 details extr: 29, plas: 2, lyso: 1 g10514.t1 details nucl: 19, cyto_nucl: 15, cyto: 9, extr: 3, mito: 1 g2203.t1 details plas: 29, nucl: 2, golg: 1 g10514.t1 details nucl: 19, cyto_nucl: 15, cyto: 9, extr: 3, mito: 1 g5502.t1 details extr: 31, lyso: 1 g5641.t1 details extr: 31, lyso: 1 g5503.t1 details extr: 29, plas: 1, mito: 1, lyso: 1 g1285.t1 details extr: 25, plas: 5, mito: 1, lyso: 1 g15153.t1 details extr: 32 g5616.t1 details extr: 31, mito: 1 g12388.t1 details extr: 25, plas: 4, mito: 2, lyso: 1 g5467.t1 details extr: 27, plas: 4, mito: 1 g3428.t1 details mito: 18, cyto: 11, extr: 2, nucl: 1 g11513.t1 details cyto: 17, cyto_nucl: 12.8333, cyto_mito: 9.83333, nucl: 7.5, E.R.: 3, mito: 1.5, plas: 1, pero: 1, golg: 1 g3679.t1 details extr: 26, mito: 2, lyso: 2, plas: 1, E.R.: 1 g5237.t1 details plas: 24, mito: 8"

In [ ]:
a = "guujb g57776.t1 ,nkjklmklmk jk g4.t1 hbhb"
#re.split(r'(g[1-9]*.t1*)', a)
#a.split(r'(g[1-9]*.t1)')

In [ ]:
import re

wolf_list = re.split(r'(g[0-9]*.t1*)', wolf_res)

result = []
for i in range(1, len(wolf_list)-1, 2):
    print(wolf_list[i], wolf_list[i+1])
    if "nucl" in wolf_list[i+1]:
        result.append(wolf_list[i])

for i in range(len(result)):
    result[i] = "g"+str(result[i])

gene_names = set()
for gene in result:
    gene_names.add(gene.split(" ")[0])

g5641.t1  details extr: 31, lyso: 1 
g15153.t1  details extr: 32 
g5641.t1  details extr: 31, lyso: 1 
g12562.t1  details extr: 30, lyso: 2 
g5616.t1  details extr: 31, mito: 1 
g15153.t1  details extr: 32 
g12562.t1  details extr: 30, lyso: 2 
g5616.t1  details extr: 31, mito: 1 
g702.t1  details extr: 29, plas: 2, lyso: 1 
g15153.t1  details extr: 32 
g14472.t1  details nucl: 28, plas: 2, cyto: 1, cysk: 1 
g4106.t1  details E.R.: 14.5, E.R._golg: 9.5, extr: 7, golg: 3.5, lyso: 3, pero: 2, plas: 1, mito: 1 
g7861.t1  details nucl: 16, cyto_nucl: 14, cyto: 8, plas: 5, pero: 1, cysk: 1, golg: 1 
g4970.t1  details plas: 32 
g10513.t1  details nucl: 20, cyto_nucl: 14.5, cyto: 7, extr: 3, E.R.: 1, golg: 1 
g11806.t1  details nucl: 18, cyto_nucl: 11.8333, mito: 5, extr: 4, cyto: 3.5, cyto_pero: 2.66667, cysk_plas: 1 
g16318.t1  details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1 
g16368.t1  details nucl: 20.5, cyto_nucl: 13, extr: 5, cyto: 4.5, E.R.: 1, golg: 1 
g3428.t1

In [ ]:
for gene in sorted(gene_names):
    print(gene)

gg10513.t1
gg10514.t1
gg11513.t1
gg11806.t1
gg11960.t1
gg14472.t1
gg16318.t1
gg16368.t1
gg2203.t1
gg3428.t1
gg5443.t1
gg5927.t1
gg7861.t1
gg8100.t1
gg8312.t1


In [ ]:
len(gene_names)

15

In [1]:
!xargs samtools faidx augustus.whole.aa < set_genes_wolf.txt > wolf.fa

/bin/bash: set_genes_wolf.txt: No such file or directory
